In [3]:
import pandas as pd
import numpy as np
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

In [4]:
data = pd.read_csv('Books.csv')
data.head()

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_5388\1537572215.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('Books.csv')


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [5]:
# Cleaning Data Function
def clean_text(text):
    cleaned = re.sub(r'[^\x00-\x7F]+', '', text)
    return cleaned.strip().lower()
# Clean the book titles
data['Book-Title'] = data['Book-Title'].apply(clean_text)

In [6]:
# Convert the 'Year-Of-Publication' column to numeric, forcing invalid values to NaN
data['Year-Of-Publication'] = pd.to_numeric(data['Year-Of-Publication'], errors='coerce')

# Drop rows where 'Year-Of-Publication' is NaN
data = data.dropna(subset=['Year-Of-Publication'])

# Filter books published in the year 2000 or later
filtered_data = data[data['Year-Of-Publication'] == 2004]

In [7]:
filtered_data.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
12,0887841740,the middle stories,Sheila Heti,2004.0,House of Anansi Press,http://images.amazon.com/images/P/0887841740.0...,http://images.amazon.com/images/P/0887841740.0...,http://images.amazon.com/images/P/0887841740.0...
26,0971880107,wild animus,Rich Shapero,2004.0,Too Far,http://images.amazon.com/images/P/0971880107.0...,http://images.amazon.com/images/P/0971880107.0...,http://images.amazon.com/images/P/0971880107.0...
117,0553584383,dead aim,IRIS JOHANSEN,2004.0,Bantam Books,http://images.amazon.com/images/P/0553584383.0...,http://images.amazon.com/images/P/0553584383.0...,http://images.amazon.com/images/P/0553584383.0...
120,0316735736,all he ever wanted: a novel,Anita Shreve,2004.0,Back Bay Books,http://images.amazon.com/images/P/0316735736.0...,http://images.amazon.com/images/P/0316735736.0...,http://images.amazon.com/images/P/0316735736.0...
243,140003180X,the kalahari typing school for men (no. 1 ladi...,ALEXANDER MCCALL SMITH,2004.0,Anchor,http://images.amazon.com/images/P/140003180X.0...,http://images.amazon.com/images/P/140003180X.0...,http://images.amazon.com/images/P/140003180X.0...


In [8]:
titles = filtered_data['Book-Title'].values
tokenizer = Tokenizer()
tokenizer.fit_on_texts(titles)
total_words = len(tokenizer.word_index) + 1

input_sequences = []
for line in titles:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Pad sequences
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# Step 5: Prepare input and label data
X = input_sequences[:, :-1]
y = input_sequences[:, -1]
# y = np.array(y)

# Convert labels to one-hot encoding
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

In [9]:
# Step 6: Build the LSTM model
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [10]:
# Step 7: Train the model
early_stop = EarlyStopping(monitor='loss', patience=3)
model.fit(X, y, epochs=50, callbacks=[early_stop], verbose=1)

# Step 8: Save the model
model.save('model_lstm.h5')

print("Model has been trained and saved as lstm_model.h5")

Epoch 1/50
887/887 [==============================] - 47s 50ms/step - loss: 7.6266 - accuracy: 0.0598
Epoch 2/50
887/887 [==============================] - 47s 53ms/step - loss: 6.9768 - accuracy: 0.0908
Epoch 3/50
887/887 [==============================] - 53s 60ms/step - loss: 6.5037 - accuracy: 0.1249
Epoch 4/50
887/887 [==============================] - 65s 73ms/step - loss: 6.0011 - accuracy: 0.1519
Epoch 5/50
887/887 [==============================] - 65s 73ms/step - loss: 5.5056 - accuracy: 0.1811
Epoch 6/50
887/887 [==============================] - 60s 67ms/step - loss: 5.0419 - accuracy: 0.2053
Epoch 7/50
887/887 [==============================] - 64s 72ms/step - loss: 4.6196 - accuracy: 0.2346
Epoch 8/50
887/887 [==============================] - 52s 59ms/step - loss: 4.2229 - accuracy: 0.2710
Epoch 9/50
887/887 [==============================] - 59s 66ms/step - loss: 3.8546 - accuracy: 0.3166
Epoch 10/50
887/887 [==============================] - 57s 64ms/step - loss: 3.498

c:\Users\LENOVO\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model has been trained and saved as lstm_model.h5


In [11]:
import json

# Save the tokenizer
tokenizer_json = tokenizer.to_json()
with open('tokenizer.json', 'w') as f:
    json.dump(tokenizer_json, f)

In [12]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import json

# Load the trained model
model = load_model('model_lstm.h5')

# Load the tokenizer
with open('tokenizer.json') as f:
    tokenizer_json = json.load(f)

from tensorflow.keras.preprocessing.text import tokenizer_from_json
tokenizer = tokenizer_from_json(tokenizer_json)

# Function to generate book title suggestions based on user input
def generate_suggestions(seed_text, num_suggestions=10):
    max_sequence_len = model.input_shape[1] + 1  # Get max sequence length from the model input shape
    suggestions = []
    
    for _ in range(num_suggestions):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict(token_list, verbose=0)
        predicted_word_index = np.argmax(predicted, axis=-1)
        predicted_word = ''
        
        for word, index in tokenizer.word_index.items():
            if index == predicted_word_index:
                predicted_word = word
                break
        
        # Append the predicted word to the seed text
        seed_text += " " + predicted_word
        suggestions.append(seed_text)
    
    return suggestions

# Example test
user_input = "adventure"
suggestions = generate_suggestions(user_input)

print("Suggestions based on input '{}':".format(user_input))
for suggestion in suggestions:
    print(suggestion)

Suggestions based on input 'adventure':
adventure in
adventure in my
adventure in my heart
adventure in my heart aiken
adventure in my heart aiken ginny
adventure in my heart aiken ginny silver
adventure in my heart aiken ginny silver hills
adventure in my heart aiken ginny silver hills trilogy
adventure in my heart aiken ginny silver hills trilogy the
adventure in my heart aiken ginny silver hills trilogy the next
